# Track Tree - Example Data Generation

----

This generates some random example data to illustrate the "track tree" visualization. The values for the two measurement tracks are generated by a pink noise generator. The tree structure is also randomly generated (in a very awkward way that allows very little control). Have fun. 

### Prep

In [1]:
### Import modules
from __future__ import division
import os, sys
import warnings
import numpy as np
import matplotlib.pyplot as plt

In [2]:
### Seed random generator
np.random.seed(5)

### Tree Generation

In [3]:
### Function to generate two branches from a stem

def make_branches(ID, IDs):
    
    # First branch
    new_ID_1 = IDs[-1] + 1
    tree[new_ID_1] = {'stem'     : ID,
                      'branches' : '_none_'}
    IDs.append(new_ID_1)
    
    # Second branch
    new_ID_2 = IDs[-1] + 1
    tree[new_ID_2] = {'stem'     : ID,
                      'branches' : '_none_'}
    IDs.append(new_ID_2)
    
    # Update stem
    tree[ID]['branches'] = [new_ID_1, new_ID_2]

In [4]:
### Generate the tree

# Parameters
max_iter = 10
p_branch = 0.55

# Start by generating the root 
tree = { 0 : {'stem'     : '_root_',
              'branches' : '_none_'} }

# Keep track of IDs
IDs = [0]

# Iterate
for iterstep in range(max_iter):
    
    # For each existing stem...
    for ID in tree.keys():
        
        # If it can create branches...
        if tree[ID]['branches'] == '_none_':
            
            # Randomly decide if it branches
            if np.random.binomial(1, p_branch) or tree[ID]['stem'] == '_root_':
                
                # Create the new branches
                make_branches(ID, IDs)
                
            # Otherwise, make it a leaf
            else: 
                tree[ID]['branches'] = ['_leaf_', '_leaf_']
                
# Clean up final leaves (in case max_iter was reached)
for ID in tree.keys():
    if tree[ID]['branches'] == '_none_':
        tree[ID]['branches'] = ['_leaf_', '_leaf_']

### Track Generation: Indices

In [5]:
### Generate length & indices of all tracks

# Parameters
min_len =  10
max_len = 100

# Function to recursively assign lenghts / indices
def generate_indices(ID):
    
    # Randomly generate length
    track_len = np.random.randint(min_len, max_len)
    
    # Generate indices for the root (count from zero)
    if tree[ID]['stem'] == '_root_':
        track_indices = np.arange(0, track_len)
    
    # Generate indices for branches/leaves (count from stem position)
    else:
        track_indices = np.arange(1, track_len+1) + tree[tree[ID]['stem']]['indices'][-1]
    
    # Add indices to tree
    tree[ID]['indices'] = track_indices
    
    # Generate indices for the branches (recursion)
    if tree[ID]['branches'][0] != '_leaf_':
        generate_indices(tree[ID]['branches'][0])
        generate_indices(tree[ID]['branches'][1])
        
# Run the function
generate_indices(0)

### Track Generation: Measurements

In [6]:
### Function to generate pink-noise
#     Adapted from Allen B. Downey,
#     github.com/AllenDowney/ThinkDSP/blob/master/code/voss.ipynb
#     I removed the pandas dependency (at the cost of speed and scaling...)

def voss(nrows, ncols=16):
    """Generates pink noise using the Voss-McCartney algorithm.
    
    nrows: number of values to generate
    rcols: number of random sources to add
    
    returns: NumPy array
    """
    
    # Set up the array
    array = np.empty((nrows, ncols))
    array.fill(np.nan)
    
    # Populate first row (first time point)
    array[0, :] = np.random.random(ncols)
    
    # Populate first columns (highest-freq generator)
    array[:, 0] = np.random.random(nrows)
    
    # Compute where changes happen and add new values
    n = nrows  # the total number of changes is nrows
    cols = np.random.geometric(0.5, n)
    cols[cols >= ncols] = 0
    rows = np.random.randint(nrows, size=n)
    array[rows, cols] = np.random.random(n)
    
    # Forward-fill the skipped nan values
    lel = np.copy(array)
    while np.any(np.isnan(array)):
        nan_r, nan_c = np.where(np.isnan(array))
        fillable = np.where(~np.isnan(array[nan_r-1, nan_c]))
        array[nan_r[fillable], nan_c[fillable]] = array[nan_r[fillable]-1, nan_c[fillable]]

    # Return the sums
    return array.sum(axis=1)

In [7]:
### Generate two measurements (at different order of magnitude)

# For each branch...
for ID in tree.keys():
    
    # For each measure & magnitude...
    for m_name, m_magnitude in zip(['measure_1','measure_2'],[1,10]):
        
        # Create measure track
        tree[ID][m_name] = voss(tree[ID]['indices'].shape[0], ncols=16) * m_magnitude

### Saving Generated Data

In [8]:
### Save the tree structure
# Files will be "ID\tStem\tBranch1\tBranch2\n"

with open("tree_struct.txt","w") as outfile:
    
    # Write header
    outfile.write("trackID\tstemID\tbranchID1\tbranchID2\n")
    
    # Iterate over branches in random order
    # (...to better approximate real experimental result lists)
    shuffled_keys = tree.keys()
    np.random.shuffle(shuffled_keys)
    for ID in shuffled_keys:
        
        # Write the output
        outfile.write("%s\t%s\t%s\t%s\n" % (ID,
                                            tree[ID]['stem'],
                                            tree[ID]['branches'][0],
                                            tree[ID]['branches'][1]))

In [9]:
### Save data tracks

for m_name in ['measure_1','measure_2']:
    
    # Create fname
    fname = "tracks_"+m_name+".txt"
        
    # Create header
    all_IDs = sorted(tree.keys())
    header = '\t'.join(["index"]+[str(ID) for ID in all_IDs])

    # Create a numpy array containing all track data
    # Note: first column is the time course index
    final_index = np.max(np.concatenate([tree[ID]['indices'] for ID in all_IDs]))
    track_array = np.zeros((final_index+1, len(all_IDs)+1))
    track_array.fill(np.nan)
    track_array[:, 0] = np.arange(final_index+1)
    for track_idx,ID in enumerate(all_IDs):
        track_array[tree[ID]['indices'], track_idx+1] = tree[ID][m_name]

    # Write the file
    fmt = ['%d'] + [' %.3f' for track_idx in range(len(all_IDs))]  # To write index as d, everything else as f
    np.savetxt(fname, track_array, fmt=fmt, delimiter='\t', header=header, comments='') 